In [1]:
import os, time, pathlib, pprint, requests, json
from datetime import datetime, timedelta

EP = os.getenv("BROWSER_ENDPOINT", "http://localhost:8004")
SCRAPED = pathlib.Path("/storage/scraped_data")

def wait_for(job_id, every=3):
    print(f"⏳ Waiting for job {job_id}...")
    while True:
        rec = requests.get(f"{EP}/jobs/{job_id}").json()
        status = rec["status"]
        if status not in {"finished", "error"}:
            print(f"\r⏱️  {rec['status_with_elapsed']}", end="")
        else:
            print(f"\n✅ {status.upper()}")
            return rec
        time.sleep(every)

def submit(task, payload):
    print(f"🚀 Submitting {task} task...")
    print(f"📝 Payload: {json.dumps(payload, indent=2)}")
    r = requests.post(f"{EP}/jobs/{task}", json=payload)
    r.raise_for_status()
    jid = r.json()["job_id"]
    print(f"🆔 Job ID: {jid}")
    return wait_for(jid)

def analyze_results(result, test_name):
    """Enhanced analysis function for the updated booking scraper with filter support."""
    print(f"\n" + "="*70)
    print(f"📊 ANALYSIS: {test_name}")
    print("="*70)
    
    if result["status"] == "error":
        print(f"❌ FAILED: {result.get('error', 'Unknown error')}")
        return
    
    if "result" not in result:
        print(f"❌ No result data found")
        pprint.pp(result)
        return
        
    res = result["result"]
    metadata = res.get("search_metadata", {})
    hotels = res.get("hotels", [])
    
    # === BASIC INFO ===
    print(f"✅ STATUS: Task completed successfully")
    print(f"🏨 HOTELS FOUND: {len(hotels)} (total_found: {metadata.get('total_found', 'N/A')})")
    print(f"📍 LOCATION: {metadata.get('location', 'N/A')}")
    print(f"📅 DATES: {metadata.get('check_in', 'N/A')} → {metadata.get('check_out', 'N/A')} ({metadata.get('nights', 'N/A')} nights)")
    print(f"🔧 METHOD: {metadata.get('extraction_method', 'N/A')}")
    print(f"📊 SCRAPE LEVEL: {metadata.get('scrape_level', 'N/A')}")
    
    if not hotels:
        print(f"❌ NO HOTELS FOUND")
        if metadata.get('total_found', 0) == 0:
            print(f"   • This could indicate filter settings are too restrictive")
            print(f"   • Or location search returned no results")
        return
    
    # === PRICING ANALYSIS ===
    hotels_with_prices = [h for h in hotels if h.get('price_per_night')]
    if hotels_with_prices:
        prices = [h['price_per_night'] for h in hotels_with_prices]
        avg_price = sum(prices) / len(prices)
        min_price = min(prices)
        max_price = max(prices)
        print(f"💰 PRICING: ${min_price}-${max_price} (avg: ${avg_price:.0f})")
        print(f"   • {len(hotels_with_prices)}/{len(hotels)} hotels have price data")
    else:
        print(f"💰 PRICING: No price data available")
        print(f"   • Metadata avg: ${metadata.get('average_price', 0)}")
    
    # === RATING ANALYSIS ===
    hotels_with_ratings = [h for h in hotels if h.get('rating')]
    if hotels_with_ratings:
        ratings = [h['rating'] for h in hotels_with_ratings]
        avg_rating = sum(ratings) / len(ratings)
        min_rating = min(ratings)
        max_rating = max(ratings)
        print(f"⭐ RATINGS: {min_rating:.1f}-{max_rating:.1f} (avg: {avg_rating:.1f}/10)")
        print(f"   • {len(hotels_with_ratings)}/{len(hotels)} hotels have rating data")
    else:
        print(f"⭐ RATINGS: No rating data available")
    
    # === DATA QUALITY ===
    completeness_scores = [h.get('data_completeness', 0) for h in hotels]
    avg_completeness = sum(completeness_scores) / len(completeness_scores) if completeness_scores else 0
    print(f"📊 DATA COMPLETENESS: {avg_completeness:.1f}% average")
    print(f"   • Success rate: {metadata.get('success_rate', 0):.1%}")
    
    # === REVIEWS ANALYSIS ===
    hotels_with_reviews = [h for h in hotels if h.get('reviews') and len(h['reviews']) > 0]
    if hotels_with_reviews:
        total_reviews = sum(len(h.get('reviews', [])) for h in hotels_with_reviews)
        avg_reviews_per_hotel = total_reviews / len(hotels_with_reviews)
        print(f"📝 REVIEWS: {total_reviews} total from {len(hotels_with_reviews)}/{len(hotels)} hotels")
        print(f"   • Avg: {avg_reviews_per_hotel:.1f} reviews per hotel")
        
        # Check extraction methods for reviews
        review_methods = set()
        for h in hotels_with_reviews:
            if h.get('extraction_method'):
                review_methods.add(h['extraction_method'])
        if review_methods:
            print(f"   • Methods: {', '.join(review_methods)}")
    else:
        # Check if review counts exist but no reviews extracted
        hotels_with_review_counts = [h for h in hotels if h.get('review_count', 0) > 0]
        if hotels_with_review_counts:
            total_review_count = sum(h.get('review_count', 0) for h in hotels_with_review_counts)
            print(f"📝 REVIEWS: 0 extracted (but {total_review_count} available)")
            print(f"   ⚠️  Review extraction may need debugging")
        else:
            print(f"📝 REVIEWS: None available or extracted")
    
    # === FILTER VALIDATION ===
    # Check if filters are working by analyzing the results
    params_str = str(result.get('params', {}))
    has_price_filters = 'min_price' in params_str or 'max_price' in params_str
    has_rating_filters = 'min_rating' in params_str
    has_amenity_filters = 'amenities' in params_str
    
    if has_price_filters or has_rating_filters or has_amenity_filters:
        print(f"🎯 FILTER ANALYSIS:")
        
        if has_price_filters and hotels_with_prices:
            prices = [h['price_per_night'] for h in hotels_with_prices]
            print(f"   • Price filter results: ${min(prices)}-${max(prices)}")
            
        if has_rating_filters and hotels_with_ratings:
            ratings = [h['rating'] for h in hotels_with_ratings]
            print(f"   • Rating filter results: {min(ratings):.1f}-{max(ratings):.1f}/10")
            
        if has_amenity_filters:
            hotels_with_amenities = [h for h in hotels if h.get('amenities')]
            if hotels_with_amenities:
                total_amenities = sum(len(h.get('amenities', [])) for h in hotels_with_amenities)
                print(f"   • Amenity data: {total_amenities} total from {len(hotels_with_amenities)} hotels")
            else:
                print(f"   • Amenity data: None found")
    
    # === LOCATION ACCURACY ===
    location_query = metadata.get('location', '').lower()
    if location_query:
        accurate_hotels = 0
        for hotel in hotels:
            address = hotel.get('address', '').lower()
            if address and any(word in address for word in location_query.split() if len(word) > 3):
                accurate_hotels += 1
        
        if accurate_hotels > 0:
            accuracy_pct = (accurate_hotels / len(hotels)) * 100
            print(f"🌍 LOCATION ACCURACY: {accurate_hotels}/{len(hotels)} hotels ({accuracy_pct:.0f}%)")
        else:
            print(f"🌍 LOCATION ACCURACY: Unable to verify (limited address data)")
    
    # === EXECUTION TIME ===
    completed_at = metadata.get('search_completed_at')
    if completed_at:
        print(f"⏱️  COMPLETED: {completed_at}")
    
    # === SAMPLE HOTEL ===
    if hotels:
        sample = hotels[0]
        print(f"\n📋 SAMPLE HOTEL:")
        print(f"   🏨 {sample.get('name', 'Unknown')}")
        print(f"   📍 {sample.get('address', 'No address')}")
        print(f"   💰 ${sample.get('price_per_night', 'N/A')}/night")
        print(f"   ⭐ {sample.get('rating', 'N/A')}/10 ({sample.get('review_count', 0)} reviews)")
        print(f"   📊 {sample.get('data_completeness', 0):.0f}% complete")
        print(f"   🔧 {sample.get('extraction_method', sample.get('source', 'Unknown method'))}")
        
        amenities = sample.get('amenities', [])
        if amenities:
            print(f"   🎯 {len(amenities)} amenities: {', '.join(amenities[:5])}{'...' if len(amenities) > 5 else ''}")

# Calculate future dates for testing
today = datetime.now()
check_in_1 = (today + timedelta(days=30)).strftime("%Y-%m-%d")
check_out_1 = (today + timedelta(days=33)).strftime("%Y-%m-%d")
check_in_2 = (today + timedelta(days=45)).strftime("%Y-%m-%d")
check_out_2 = (today + timedelta(days=47)).strftime("%Y-%m-%d")
check_in_3 = (today + timedelta(days=60)).strftime("%Y-%m-%d")
check_out_3 = (today + timedelta(days=64)).strftime("%Y-%m-%d")

print(f"📅 Test dates calculated:")
print(f"   Test 1: {check_in_1} to {check_out_1}")
print(f"   Test 2: {check_in_2} to {check_out_2}")
print(f"   Test 3: {check_in_3} to {check_out_3}")

📅 Test dates calculated:
   Test 1: 2025-10-03 to 2025-10-06
   Test 2: 2025-10-18 to 2025-10-20
   Test 3: 2025-11-02 to 2025-11-06


In [2]:
# Test NEW SCRAPE LEVEL SYSTEM - FOCUSED ON FIXES
print("🧪 TESTING FIXES FOR REPORTED ISSUES")
print("="*70)
print("🎯 VALIDATING:")
print("   • Level 1: Rating extraction working")  
print("   • All levels: Reviewer names NOT 'Wonderful'")
print("   • All levels: Clear differences between levels")
print("="*70)

def test_scrape_level_focused(level, location="Dubai"):
    """Test a specific scrape level with focus on reported issues"""
    payload = {
        "location": location,
        "scrape_level": level,
        "max_results": 3  # Small for focused testing
    }
    
    print(f"\n🔍 Testing Scrape Level {level}")
    result = submit("booking", payload)
    
    if result["status"] == "finished":
        res = result["result"]
        hotels = res.get("hotels", [])
        metadata = res.get("search_metadata", {})
        
        print(f"📊 Level {level} Results:")
        print(f"   🔧 Method: {metadata.get('extraction_method')}")
        print(f"   🏨 Hotels: {len(hotels)}")
        
        # CHECK 1: RATINGS (Critical for Level 1)
        hotels_with_ratings = [h for h in hotels if h.get('rating')]
        if level == 1:
            if hotels_with_ratings:
                ratings = [h['rating'] for h in hotels_with_ratings]
                avg_rating = sum(ratings) / len(ratings)
                print(f"   ⭐ RATINGS: ✅ {len(hotels_with_ratings)}/{len(hotels)} hotels have ratings (avg: {avg_rating:.1f})")
                print(f"      Sample ratings: {ratings}")
            else:
                print(f"   ⭐ RATINGS: ❌ NO RATINGS FOUND - LEVEL 1 FAILED")
        else:
            if hotels_with_ratings:
                print(f"   ⭐ RATINGS: ✅ {len(hotels_with_ratings)}/{len(hotels)} hotels")
            else:
                print(f"   ⭐ RATINGS: ❌ None found")
        
        # CHECK 2: REVIEWS AND REVIEWER NAMES
        hotels_with_reviews = [h for h in hotels if h.get('reviews')]
        if hotels_with_reviews:
            total_reviews = sum(len(h.get('reviews', [])) for h in hotels_with_reviews)
            print(f"   📝 REVIEWS: ✅ {total_reviews} from {len(hotels_with_reviews)} hotels")
            
            # CHECK FOR "Wonderful" NAMES (the critical issue)
            bad_names = []
            good_names = []
            all_reviewer_names = []
            
            for hotel in hotels_with_reviews:
                for review in hotel.get('reviews', []):
                    reviewer_name = review.get('reviewer_name', '')
                    if reviewer_name:
                        all_reviewer_names.append(reviewer_name)
                        if reviewer_name.lower() in ['wonderful', 'excellent', 'amazing', 'good', 'bad']:
                            bad_names.append(reviewer_name)
                        else:
                            good_names.append(reviewer_name)
            
            if bad_names:
                print(f"   👤 REVIEWER NAMES: ❌ FOUND BAD NAMES: {set(bad_names)}")
                print(f"      🚨 CRITICAL ISSUE NOT FIXED!")
            elif good_names:
                print(f"   👤 REVIEWER NAMES: ✅ All look valid (no 'Wonderful' found)")
                print(f"      Sample names: {good_names[:5]}")
            else:
                print(f"   👤 REVIEWER NAMES: ⚠️ No reviewer names found")
        else:
            expected_reviews = level >= 3
            if expected_reviews:
                print(f"   📝 REVIEWS: ❌ Expected reviews for Level {level} but none found")
            else:
                print(f"   📝 REVIEWS: ✅ None expected for Level {level}")
                
        return result
    else:
        print(f"❌ Level {level} FAILED: {result.get('error')}")
        return None

# Test each scrape level with focused analysis
print("\n🎯 Testing all scrape levels with Dubai...")
results = {}
for level in [1, 2, 3, 4]:
    results[level] = test_scrape_level_focused(level)
    time.sleep(2)  # Brief pause between tests

print("\n" + "="*70)
print("📋 ISSUE-FOCUSED COMPARISON")
print("="*70)

levels_desc = {
    1: "QUICK SEARCH: Should have RATINGS",
    2: "HOTEL PAGES: No reviews but full data", 
    3: "BASIC REVIEWS: 2-5 reviews with REAL names",
    4: "DEEP REVIEWS: 10+ reviews with REAL names"
}

success_count = 0
total_issues = 0

for level in [1, 2, 3, 4]:
    result = results.get(level)
    print(f"\nLevel {level} ({levels_desc[level]}):")
    
    if result and result["status"] == "finished":
        res = result["result"]
        hotels = res.get("hotels", [])
        
        # Check Level 1 rating requirement
        if level == 1:
            hotels_with_ratings = [h for h in hotels if h.get('rating')]
            if hotels_with_ratings:
                print(f"   ✅ RATINGS: {len(hotels_with_ratings)}/{len(hotels)} hotels")
                success_count += 1
            else:
                print(f"   ❌ RATINGS: MISSING - CRITICAL FAILURE")
                total_issues += 1
        
        # Check reviewer name quality for review levels
        if level >= 3:
            hotels_with_reviews = [h for h in hotels if h.get('reviews')]
            if hotels_with_reviews:
                bad_names = []
                for hotel in hotels_with_reviews:
                    for review in hotel.get('reviews', []):
                        reviewer_name = review.get('reviewer_name', '')
                        if reviewer_name and reviewer_name.lower() in ['wonderful', 'excellent', 'amazing']:
                            bad_names.append(reviewer_name)
                
                if bad_names:
                    print(f"   ❌ REVIEWER NAMES: Found bad names: {set(bad_names)}")
                    total_issues += 1
                else:
                    print(f"   ✅ REVIEWER NAMES: All valid (no 'Wonderful')")
                    success_count += 1
            else:
                print(f"   ⚠️ REVIEWS: None found for Level {level}")
        
        # Check level differentiation
        total_reviews = sum(len(h.get('reviews', [])) for h in hotels)
        avg_completeness = sum(h.get('data_completeness', 0) for h in hotels) / len(hotels) if hotels else 0
        
        print(f"   📊 Reviews: {total_reviews}, Completeness: {avg_completeness:.1f}%")
        
    else:
        print(f"   ❌ EXECUTION FAILED")
        total_issues += 1

# FINAL VERDICT
print("\n" + "="*70)
print("🎯 FIX VALIDATION RESULTS")
print("="*70)

if total_issues == 0:
    print("🎉 ALL ISSUES FIXED!")
    print("✅ Level 1 ratings working")
    print("✅ No 'Wonderful' reviewer names found")
    print("✅ All levels working with clear differences")
else:
    print(f"⚠️  {total_issues} issues still present:")
    print("❌ Some critical problems not yet resolved")
    print("🔧 Need to continue iterating on fixes")

print(f"\n📊 Success Rate: {success_count}/{success_count + total_issues} checks passed")
print("\n✅ Focused testing completed!")

🧪 TESTING FIXES FOR REPORTED ISSUES
🎯 VALIDATING:
   • Level 1: Rating extraction working
   • All levels: Reviewer names NOT 'Wonderful'
   • All levels: Clear differences between levels

🎯 Testing all scrape levels with Dubai...

🔍 Testing Scrape Level 1
🚀 Submitting booking task...
📝 Payload: {
  "location": "Dubai",
  "scrape_level": 1,
  "max_results": 3
}
🆔 Job ID: 78462e8a4468466ea664f9945750077d
⏳ Waiting for job 78462e8a4468466ea664f9945750077d...
⏱️  running 6s
✅ FINISHED
📊 Level 1 Results:
   🔧 Method: level_1_quick_search
   🏨 Hotels: 3
   ⭐ RATINGS: ✅ 3/3 hotels have ratings (avg: 8.8)
      Sample ratings: [8.8, 9.0, 8.6]
   📝 REVIEWS: ✅ None expected for Level 1

🔍 Testing Scrape Level 2
🚀 Submitting booking task...
📝 Payload: {
  "location": "Dubai",
  "scrape_level": 2,
  "max_results": 3
}
🆔 Job ID: 3c68b79577cc4700a31e6fabdee1c60a
⏳ Waiting for job 3c68b79577cc4700a31e6fabdee1c60a...
⏱️  running 24s
✅ FINISHED
📊 Level 2 Results:
   🔧 Method: level_2_full_data
   🏨 Ho